# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

from api_keys import g_key

### Store Part 1 Dataframe

In [2]:
city_df = pd.read_csv('../cities.csv')
city_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Vaini,-21.2000,-175.2000,78.96,78,40,10.36,TO,1636843144
1,1,Cherskiy,68.7500,161.3000,17.35,96,100,4.68,RU,1636843144
2,2,Hualmay,-11.0964,-77.6139,65.39,80,30,9.48,PE,1636843145
3,3,Santa Rosalía,27.3167,-112.2833,77.54,35,78,3.65,MX,1636842672
4,4,Rikitea,-23.1203,-134.9692,73.42,73,50,12.68,PF,1636842672


### Humidity Heatmap

In [3]:
gmaps.configure(api_key=g_key)
city_lat_lng = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

In [8]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(city_lat_lng, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [10]:
weather_data = city_df.loc[(city_df['Wind Speed']<= 10) & (city_df['Cloudiness'] == 0) & 
                                    (city_df['Max Temp']<=80) & (city_df['Max Temp'] >= 70)]
weather_data.dropna()
weather_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
51,51,Teseney,15.1100,36.6575,79.38,15,0,5.50,ER,1636842909
89,89,Ballina,-28.8667,153.5667,72.79,45,0,8.05,AU,1636842986
148,148,Aswān,24.0934,32.9070,72.70,64,0,4.61,EG,1636843185
291,291,San Patricio,28.0170,-97.5169,75.61,30,0,6.04,US,1636843254
292,292,Salalah,17.0151,54.0924,77.09,50,0,2.30,OM,1636843254


### Hotel Map

In [11]:
hotel_df = weather_data.loc[:,:]
hotel_df['Hotel Name'] = ""

target_type = "hotel"
radius = 5000

params = {
    "keyword" : "hotel",
    "types": target_type,
    "radius": radius,
    "key": g_key
}


base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


for index, row in hotel_df.iterrows():
    city_name = row["City"]

    params["location"] = f"{row['Lat']}, {row['Lng']}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    time.sleep(1)

print("DONE")

/Users/jordanjaner/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Retrieving Results for Index 51: Teseney.
Closest hotel in Teseney is Ferait Hotel.
------------


/Users/jordanjaner/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Retrieving Results for Index 89: Ballina.
Closest hotel in Ballina is Ramada Hotel & Suites by Wyndham Ballina Byron.
------------
Retrieving Results for Index 148: Aswān.
Closest hotel in Aswān is Sofitel Legend Old Cataract Aswan.
------------
Retrieving Results for Index 291: San Patricio.
Closest hotel in San Patricio is Americas Best Value Inn Sinton.
------------
Retrieving Results for Index 292: Salalah.
Closest hotel in Salalah is Muscat International Hotel plaza Salalah.
------------
Retrieving Results for Index 381: Pasni.
Closest hotel in Pasni is Juddi Hotel.
------------
Retrieving Results for Index 472: Barentu.
Missing field/result... skipping.
------------
Retrieving Results for Index 538: Coolum Beach.
Closest hotel in Coolum Beach is Coolum Beach Hotel.
------------
DONE


In [12]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
51,51,Teseney,15.1100,36.6575,79.38,15,0,5.50,ER,1636842909,Ferait Hotel
89,89,Ballina,-28.8667,153.5667,72.79,45,0,8.05,AU,1636842986,Ramada Hotel & Suites by Wyndham Ballina Byron
148,148,Aswān,24.0934,32.9070,72.70,64,0,4.61,EG,1636843185,Sofitel Legend Old Cataract Aswan
291,291,San Patricio,28.0170,-97.5169,75.61,30,0,6.04,US,1636843254,Americas Best Value Inn Sinton
292,292,Salalah,17.0151,54.0924,77.09,50,0,2.30,OM,1636843254,Muscat International Hotel plaza Salalah
381,381,Pasni,25.2631,63.4710,76.53,37,0,7.00,PK,1636843298,Juddi Hotel
472,472,Barentu,15.1139,37.5928,73.00,18,0,1.99,ER,1636843342,
538,538,Coolum Beach,-26.5333,153.1000,75.11,42,0,2.44,AU,1636843374,Coolum Beach Hotel


In [13]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
markers = gmaps.marker_layer(locations)

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))